In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, date
import time

In [15]:
# We have 60 CSV files
#    30 tickers, 1 file containing 2012 to 2018, 1 file containing 2018 to 2020
raw_data_dir = 'main/data_raw'
# raw_train_data = [pd.read_csv(f'{raw_data_dir}/{i}') for i in os.listdir(raw_data_dir) if '2012' in i]
# raw_test_data = [pd.read_csv(f'{raw_data_dir}/{i}') for i in os.listdir(raw_data_dir) if '2020' in i]

# intermediate_train_data = []
# intermediate_test_data = []

train_fnames = [f'{raw_data_dir}/{i}' for i in os.listdir(raw_data_dir) if '2012' in i]
test_fnames = [f'{raw_data_dir}/{i}' for i in os.listdir(raw_data_dir) if '2020' in i]
train_data = pd.read_csv(train_fnames[0])
train_data = train_data[train_data.Volume != 0]
test_data = pd.read_csv(test_fnames[0])
test_data = test_data[test_data.Volume != 0]

train_data

,Timestamp,Open,High,Low,Close,Volume
0,2012-01-03,31.799999,32.000000,31.559999,30.204050,11726370.0
1,2012-01-04,32.000000,32.000000,31.600000,30.166059,9270400.0
2,2012-01-05,31.799999,31.959999,31.360001,29.900114,15810575.0
3,2012-01-06,30.920000,31.500000,30.780001,29.349222,15174110.0
4,2012-01-09,30.920000,31.180000,30.059999,29.482199,14090720.0
...,...,...,...,...,...,...
1473,2017-12-22,403.799988,405.799988,400.799988,402.698151,16146080.0
1474,2017-12-27,407.799988,407.799988,401.000000,398.133331,16680601.0
1475,2017-12-28,404.000000,408.200012,402.200012,405.079834,11662053.0
1476,2017-12-29,408.000000,408.000000,403.399994,402.896637,16601658.0


In [27]:
def get_lobf(ys: pd.DataFrame):
    xs = np.arange(ys.shape[0])
    m = (xs.mean()*ys.mean() - (xs*ys).mean())/(xs.mean()*xs.mean() - (xs*xs).mean())
    b = ys.mean() - m*xs.mean()
    reg_line = [m*x+b for x in xs]
    y_bar = ys.mean()
    r2 = 1 - sum((ys[i]-reg_line[i])**2 for i in range(len(ys)))/sum((y-y_bar)**2 for y in ys)
    return m, r2 if not np.isnan(r2) else 0

(0.07950109126645183, 0.7917697265147592)
(0.07685819971967213, 0.737708158963716)
(0.1052682725950916, 0.38838897484714774)
(0.05795519692557605, 0.06483490041429785)
(-0.15454483032226204, 0.1758236989080736)


C:\Users\calvi\AppData\Local\Temp\ipykernel_8580\2817384411.py:10: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  train_data.Close[:180]


In [28]:

history = 180
bins = [180, 90, 30, 7, 3]
res = []


for i in range(180, train_data.shape[0]-1):
    print(f'{i}\r', end='')
    train_temp = train_data[['Close', 'Volume']][i-history:i]

    close_slope_and_r2 = [get_lobf(train_temp.Close.values[-j:]) for j in bins]
    close_slope = [i[0] for i in close_slope_and_r2]
    close_r2 = [i[1] for i in close_slope_and_r2]
    vol_slope_and_r2 = [get_lobf(train_temp.Volume.values[-j:]) for j in bins]
    vol_slope = [i[0] for i in vol_slope_and_r2]
    vol_r2 = [i[1] for i in vol_slope_and_r2]

    close_today = [train_data.Close.values[i]]

    close_change_tomorrow = [train_data.Close.values[i+1]]

    total_row = close_slope + close_r2 + vol_slope + vol_r2 + close_today + close_change_tomorrow
    res.append(total_row)

output = pd.DataFrame(res, columns = ['close-slope-180', 'close-slope-90', 'close-slope-30', 'close-slope-7', 'close-slope-3',
                            'close-r2-180', 'close-r2-90', 'close-r2-30', 'close-r2-7', 'close-r2-3',
                            'vol-slope-180', 'vol-slope-90', 'vol-slope-30', 'vol-slope-7', 'vol-slope-3',
                            'vol-r2-180', 'vol-r2-90', 'vol-r2-30', 'vol-r2-7', 'vol-r2-3',
                            'close_today', 'close_tomorrow'
                            ])

C:\Users\calvi\AppData\Local\Temp\ipykernel_8580\2817384411.py:7: RuntimeWarning: invalid value encountered in double_scalars
  r2 = 1 - sum((ys[i]-reg_line[i])**2 for i in range(len(ys)))/sum((y-y_bar)**2 for y in ys)


In [30]:
output.to_csv('asdf.csv')

In [31]:
from sklearn.decomposition import PCA

In [36]:

output_norm = (output - output.mean())/output.std()
pca = PCA(n_components=output_norm.shape[1])
pca.fit(output_norm)

loadings = pd.DataFrame(pca.components_.T,
columns=['PC%s' % _ for _ in range(len(output_norm.columns))],
index=output.columns)
print(loadings)


                      PC0       PC1       PC2       PC3       PC4       PC5  \
close-slope-180  0.422540  0.120212 -0.060153  0.113268 -0.101696 -0.088670   
close-slope-90   0.422418  0.078991 -0.075070  0.044135 -0.084791  0.003029   
close-slope-30   0.300979 -0.160160  0.029084 -0.231690  0.236185  0.217659   
close-slope-7    0.129404 -0.301162  0.491326 -0.055936 -0.106937  0.061845   
close-slope-3    0.076958 -0.231485  0.478227 -0.020514 -0.187343  0.056984   
close-r2-180     0.247589  0.232870 -0.270732  0.169975 -0.310565 -0.009649   
close-r2-90      0.303087 -0.026596 -0.231859  0.063578 -0.118488 -0.025079   
close-r2-30      0.068413 -0.042289 -0.087458 -0.486752  0.235128  0.437860   
close-r2-7      -0.002779 -0.176487  0.222033  0.405791  0.055055 -0.031931   
close-r2-3      -0.018051 -0.049445  0.080739  0.195368 -0.155120  0.204995   
vol-slope-180    0.105525  0.341840  0.372218 -0.072033 -0.080145  0.214135   
vol-slope-90     0.024221  0.457928  0.301578  0.069

In [35]:
pca.

{'copy': True,
 'iterated_power': 'auto',
 'n_components': 5,
 'n_oversamples': 10,
 'power_iteration_normalizer': 'auto',
 'random_state': None,
 'svd_solver': 'auto',
 'tol': 0.0,
 'whiten': False}